In [1]:
# install asteroid
!git clone https://github.com/asteroid-team/asteroid
%cd asteroid
!pip install -e .
!pip install asteroid

!pip install pytorch-lightning
!pip install noisereduce

fatal: destination path 'asteroid' already exists and is not an empty directory.
/content/asteroid
Obtaining file:///content/asteroid
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: asteroid
    Found existing installation: asteroid 0.5.2
    Uninstalling asteroid-0.5.2:
      Successfully uninstalled asteroid-0.5.2
  Running setup.py develop for asteroid
  Using cached asteroid-0.5.2-py3-none-any.whl (246 kB)


In [2]:
import torch
from torch import optim
from pytorch_lightning import Trainer
from asteroid.models import ConvTasNet
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper
from asteroid.data import LibriMix
from asteroid.engine import System
from torch.utils.data import DataLoader
import noisereduce as nr

# to enable cuda, you have to do a manual toggle in the settings
# menu 'runtime->change runtime->hardware accelerator->gpu'
torch.cuda.is_available()

True

In [3]:
# import custom data set from py in google drive
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive')
from custom_dataset_sandbox import CustomMixture 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
csv_path = '/content/drive/MyDrive/output2/'
%cd /content/drive/MyDrive/output2/
train_loader = DataLoader(CustomMixture(csv_path, "train_metadata.csv"), num_workers=8, batch_size=2)
val_loader = DataLoader(CustomMixture(csv_path, "val_metadata.csv"), num_workers=8, batch_size=2)

/content/drive/MyDrive/output2


In [5]:
# re-instantiate the class objects
model = ConvTasNet(n_src=2)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx") # PITLossWrapper works with any loss function.

# access the .ckpt file from the drive
checkpoint = torch.load('/content/drive/MyDrive/output2/lightning_logs/version_5/checkpoints/epoch=39-step=89799.ckpt')

In [6]:
''' load the stored data into the class objects '''
print(checkpoint.keys())

# adjust 'state_dict' to fit what model requires
model_state_dict = {}
for key in checkpoint['state_dict'].keys():
  model_state_dict[key[6:]] = checkpoint['state_dict'].get(key)
# print(checkpoint['state_dict'].keys())
# print(model_state_dict.keys())
model.load_state_dict(model_state_dict)

# adjust 'optimizer_states' to fit what optimizer requires
# optimizer is expecting a dict, but 'optimizer_states' is a list...
# the list is of len() 1, passing [0] seems to fix everything
optimizer.load_state_dict(checkpoint['optimizer_states'][0])
# print(len(checkpoint['optimizer_states']))

epoch = checkpoint['epoch']

# I am electing to leave this line commented out because:
# 1. there is no dict key 'loss'
# 2. unlike model and optimizer, this class object is not being edited,
#    but overwritten
# loss = checkpoint['loss']

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'training_config'])


In [7]:
# eval mode
model.eval()

import librosa
# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output2/mixture/mixtureclip3800.wav') 
# prediction = model(torch.from_numpy(audiofile))

# import soundfile as sf

# audio1 = nr.reduce_noise(y=prediction.detach().numpy()[0], sr=sr_rate)
# audio2 = nr.reduce_noise(y=prediction.detach().numpy()[1], sr=sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_3v_3800_wav1.wav', audio1, sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_3v_3800_wav2.wav', audio2, sr_rate)

# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output2/mixture/mixtureclip188.wav') 
# prediction = model(torch.from_numpy(audiofile))

# audio1 = nr.reduce_noise(y=prediction.detach().numpy()[0], sr=sr_rate)
# audio2 = nr.reduce_noise(y=prediction.detach().numpy()[1], sr=sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_2v_188_wav1.wav', audio1, sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_2v_188_wav2.wav', audio2, sr_rate)

# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output2/mixture/mixtureclip1827.wav') 
# prediction = model(torch.from_numpy(audiofile))

# audio1 = nr.reduce_noise(y=prediction.detach().numpy()[0], sr=sr_rate)
# audio2 = nr.reduce_noise(y=prediction.detach().numpy()[1], sr=sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_1v_1827_wav1.wav', audio1, sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_1v_1827_wav2.wav', audio2, sr_rate)

# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output2/mixture/mixtureclip670.wav') 
# prediction = model(torch.from_numpy(audiofile))

# audio1 = nr.reduce_noise(y=prediction.detach().numpy()[0], sr=sr_rate)
# #audio2 = nr.reduce_noise(y=prediction.detach().numpy()[1], sr=sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_4v_670_wav1.wav', audio1, sr_rate)
# sf.write('/content/drive/MyDrive/output2/prediction_wavs/i1p2e39_4v_670_wav2orig.wav', prediction.detach().numpy()[1], sr_rate)


In [8]:

import soundfile as sf
audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output2/other_audio/Recording0004.wav') 
#audiofile = audiofile[0:88400*2]
prediction = model(torch.from_numpy(audiofile))


sf.write('/content/drive/MyDrive/output2/prediction_wavs/group_wav1.wav', prediction.detach().numpy()[0], sr_rate)
sf.write('/content/drive/MyDrive/output2/prediction_wavs/group_wav2.wav', prediction.detach().numpy()[1], sr_rate)
audio1 = nr.reduce_noise(y=prediction.detach().numpy()[0], sr=sr_rate)
audio2 = nr.reduce_noise(y=prediction.detach().numpy()[1], sr=sr_rate)
sf.write('/content/drive/MyDrive/output2/prediction_wavs/group_wav1nr.wav', audio1, sr_rate)
sf.write('/content/drive/MyDrive/output2/prediction_wavs/group_wav2nr.wav', audio2, sr_rate)